In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [3]:
# load data Bitcoin từ tệp csv
data = pd.read_csv('normalized-data.csv')
data

,time,close,ema2,ema9,ema25,ema97,volume
0,1577836800000,7176.26,7180.588135,7185.190592,7185.647417,7218.553299,1037.337
1,1577837700000,7172.36,7175.102712,7182.624474,7184.625308,7217.610579,707.833
2,1577838600000,7174.83,7174.920904,7181.065579,7183.871823,7216.737506,325.246
3,1577839500000,7171.55,7172.673635,7179.162463,7182.923990,7215.815312,378.633
4,1577840400000,7186.60,7181.957878,7180.649971,7183.206760,7215.219081,555.389
...,...,...,...,...,...,...,...
115194,1681511400000,30410.80,30411.070887,30409.768169,30390.689758,30456.936083,1418.447
115195,1681512300000,30416.20,30414.490296,30411.054535,30392.652084,30456.104735,1238.588
115196,1681513200000,30413.20,30413.630099,30411.483628,30394.232693,30455.229128,1672.811
115197,1681514100000,30402.50,30406.210033,30409.686902,30394.868640,30454.153023,3117.508


In [4]:
# chuẩn hóa dữ liệu trong khoảng [0,1] sử dụng minmaxscaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(
    data[['close', 'ema2', 'ema9', 'ema25', 'ema97', 'volume']].values)
# len(scaled_data)# có 115199 bản ghi
train_data = scaled_data[:int(len(scaled_data)*0.8)]
test_data = scaled_data[int(len(scaled_data)*0.8):]
def create_sequences(data, seq_length=672, output_length= 96):
    xs = []
    ys = []
    for i in range(len(data)-seq_length-output_length):
        x = data[i:(i+seq_length)]
        y = data[(i+seq_length):(i+seq_length+output_length)]
        ema2_predict = []
        for i in y:
            ema2_predict.append(i[1])
        xs.append(x)
        ys.append(ema2_predict)
    return np.array(xs), np.array(ys)
train_x, train_y = create_sequences(train_data)
test_x, test_y = create_sequences(test_data)
# kiểm tra kích thước của tập dữ liệu
print('in ra kích thước của tập dữ liệu')
print(train_x.shape) #(91391, 672, 6)
print(train_y.shape) #(91391, 96, 6)
print(test_x.shape) #(22272, 672, 6)
print(test_y.shape) #(22272, 96, 6)
# chuyển đổi dữ liệu thành các tensor
train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).float()
test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).float()
# tạo dataloader
from torch.utils.data import TensorDataset, DataLoader
train_data = TensorDataset(train_x, train_y)
test_data = TensorDataset(test_x, test_y)

# tạo dataloader
batch_size = 64
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)
# kiểm tra kích thước của tập dữ liệu
# print(len(train_loader)) #1428
# print(len(test_loader)) #348




in ra kích thước của tập dữ liệu
(91391, 672, 6)
(91391, 96)
(22272, 672, 6)
(22272, 96)


In [ ]:
import torch
from torch import nn

class Transformer(nn.Module):
    def __init__(self, d_model=64, nhead=8, num_encoder_layers=6, num_decoder_layers=6):
        super(Transformer, self).__init__()

        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead),
            num_encoder_layers,
        )
        self.memory = torch.rand(2, 672, 64)
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead),
            num_decoder_layers,
        )
        self.linear = nn.Linear(d_model, 96) # linear not sequence

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x,self.memory)
        x = self.linear(x)
        return x

model = Transformer(d_model=64, nhead=8, num_encoder_layers=6, num_decoder_layers=6)

# Khởi tạo đầu vào
x = torch.randn((3, 672, 64))

# Tiến hành forward propagation
output = model(x)

# In đầu ra
print(output.shape)

In [2]:
# Load dữ liệu Bitcoin từ tệp CSV
data = pd.read_csv('normalized-data.csv')

# Chuẩn hóa dữ liệu vào khoảng [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(
    data[['close', 'ema2', 'ema9', 'ema25', 'ema97', 'volume']].values)
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data = scaled_data[:800]
test_data = scaled_data[800:]

# Hàm tạo batch từ dữ liệu chuỗi thời gian


def create_sequences(data, seq_length, output_length = 92):
    sequences = []
    for i in range(len(data) - seq_length - output_length):
        sequence = data[i:i + seq_length]
        closeList = []
        ema2List = []
        ema9List = []
        ema25List = []
        ema97List = []
        volumeList = []
        ema2ListOutput = []
        for k in range(len(sequence)):
            closeList.append(sequence[k][0])
            ema2List.append(sequence[k][1])
            ema9List.append(sequence[k][2])
            ema25List.append(sequence[k][3])
            ema97List.append(sequence[k][4])
            volumeList.append(sequence[k][5])
        input_matrix = np.stack([closeList, ema9List, ema25List, ema97List, volumeList], axis=1)
        for k in range(i + seq_length, i + seq_length + output_length):
            ema2ListOutput.append(data[k][1])
        sequences.append((input_matrix, ema2ListOutput))
    return sequences

# Định nghĩa kiến trúc mô hình Transformer
# Định nghĩa kiến trúc mô hình Transformer
class Transformer(nn.Module):
    def __init__(self, d_model=64, nhead=8, num_encoder_layers=6, num_decoder_layers=6):
        super(Transformer, self).__init__()

        self.encoder = nn.TransformerEncoder( # -1, 672, 6
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead),
            num_encoder_layers,
        )
        self.memory = torch.rand(1, 672, 6)
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead),
            num_decoder_layers,
        )
        self.linear = nn.Linear(d_model, 6)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x,self.memory)
        x = self.linear(x)
        return x

model = Transformer(d_model=64, nhead=8, num_encoder_layers=6, num_decoder_layers=6)
# Cài đặt các siêu tham số
batch_size = 64
seq_length = 128
output_length = 92
num_epochs = 100
learning_rate = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Khởi tạo mô hình và bộ tối ưu
model = Transformer().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
# Chuyển đổi dữ liệu huấn luyện và tạo batch
train_sequences = create_sequences(train_data, seq_length=644)
# print(train_sequences[0])
train_inputs = torch.tensor(
    np.array([seq for seq, _ in train_sequences]), dtype=torch.float32)
train_targets = torch.tensor(
    np.array([target for _, target in train_sequences]), dtype=torch.float32)
train_dataset = torch.utils.data.TensorDataset(
    train_inputs.transpose(1, 2), train_targets)  # Chuyển vị train_inputs
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True)

#Huấn luyện mô hình
num_epochs = 100
model.train()
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_inputs.unsqueeze(2))
        loss = criterion(outputs.squeeze(), batch_targets)
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Đánh giá mô hình trên tập kiểm tra
model.eval()
test_sequences = create_sequences(test_data, seq_length=10)
test_inputs = torch.tensor([seq for seq, _ in test_sequences], dtype=torch.float32)
test_targets = torch.tensor([target for _, target in test_sequences], dtype=torch.float32)
with torch.no_grad():
    test_outputs = model(test_inputs.unsqueeze(2))
test_loss = criterion(test_outputs.squeeze(), test_targets)
print(f'Test Loss: {test_loss.item():.4f}')